# Caso Práctico  - Pamela Arico

## 1. Frases de ejemplo

Para iniciar con el caso práctico es necesario construir un corpus que contenga una variedad de frases que permitan generar una gramática lo más general posible.

In [17]:
with open('food_orders.txt', 'w') as f: 
    f.write('''Quiero una pizza con jamón.
    Quiero una pizza con queso y jamón.
    Quiero 4 hamburguesas con tocino.
    Quiero una tortilla y una cerveza.
    Me pones un pollo y una ensalada.
    Quiero una paella.
    Quiero un bocadillo.
    Quiero una pizza.
    Ponme 3 sopas.
    Quiero un filete.
    Por favor, me das tres tacos.
    Me puedes ayudar con una orden de papitas y camotes fritos.
    Ayudeme con cuatro hamburguesas con cebollas, mayonessa y mostaza.
    Por favor dos hotdogs con salsa de piña.
    Quisiera 10 alitas bbq.
    Por favor, me puedes ayudar con una ensalada con pollo.
    Seis bolones con queso.
    Quiero 12 tortillas.
    Quiero una Lasagna.
    Quiero 3 bocadillos de anchoas y 2 pizzas.
    Quisiera pedir una hamburguesa''')

In [14]:
from nltk.tokenize import sent_tokenize

def load_corpus(txt_file):
    with open(txt_file) as f: 
        data = str(f.read())
    sentences = sent_tokenize(data)
    return sentences

In [18]:
food_orders = load_corpus('food_orders.txt')
food_orders[:10]

['Quiero una pizza con jamón.',
 'Quiero una pizza con queso y jamón.',
 'Quiero 4 hamburguesas con tocino.',
 'Quiero una tortilla y una cerveza.',
 'Me pones un pollo y una ensalada.',
 'Quiero una paella.',
 'Quiero un bocadillo.',
 'Quiero una pizza.',
 'Ponme 3 sopas.',
 'Quiero un filete.']

## 2. Tagger en español

### Tagger con NLTK

El siguiente paso consiste en entrenar un tagger en español. Para ello se va a utilizar el corpus **cess_esp** disponible en la librería de nltk.

In [4]:
import nltk
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
from nltk.tag.hmm import HiddenMarkovModelTagger

In [5]:
cess_sents = cess.tagged_sents()        # Obtengo las oraciones ya etiquetadas del corpus
train = int(len(cess_sents)*90/100)     # Obtengo los indices para separar el set de entrenamiento / prueba

Se van a entrenar distintos tipos de taggers para posteriormente seleccionar el que mayor presición presente.

In [6]:
default_tagger = DefaultTagger('ncms000')
unigram_tagger = UnigramTagger(cess_sents[:train], backoff=default_tagger)
bigram_tagger = BigramTagger(cess_sents[:train], backoff=unigram_tagger)
trigram_tagger = TrigramTagger(cess_sents[:train], backoff=bigram_tagger)
hmm_tagger = HiddenMarkovModelTagger.train(cess_sents[:train])

In [8]:
print(f'Default Tagger: {default_tagger.evaluate(cess_sents[train:])}')
print(f'Unigram Tagger: {unigram_tagger.evaluate(cess_sents[train:])}')
print(f'Bigram Tagger: {bigram_tagger.evaluate(cess_sents[train:])}')
print(f'Trigram Tagger: {trigram_tagger.evaluate(cess_sents[train:])}')
print(f'HMM Tagger: {hmm_tagger.evaluate(cess_sents[train:])}')

Default Tagger: 0.0638236977676016
Unigram Tagger: 0.8218374356038924
Bigram Tagger: 0.8346451058958214
Trigram Tagger: 0.8341442472810532
HMM Tagger: 0.8490269032627361


En los resultados se puede observar que el tagger con mayor presición es el **hmm tagger**. Por lo tanto, se va a realizar una prmera prueba con una de las oraciones del corpus de ordenes de comida.

In [9]:
tokens = nltk.word_tokenize(food_orders[0])
tokens

['Quiero', 'una', 'pizza', 'con', 'jamón', '.']

In [10]:
hmm_tagger.tag(tokens)

[('Quiero', 'sps00'),
 ('una', 'di0fs0'),
 ('pizza', 'ncfs000'),
 ('con', 'sps00'),
 ('jamón', 'ncms000'),
 ('.', 'Fp')]

## 3. Regex parser

In [11]:
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *
from nltk import Tree
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
import re

A continuación se realiza una prueba para verificar los tags que se asignan a cada una de las palabras y así poder construir la gramática correspondiente.

In [19]:
for order in food_orders:
    tokens = nltk.word_tokenize(order)      # se separa en tokens cada orden
    tagged = hmm_tagger.tag(tokens)         # se asigna el pos tag correspondiente a cada token
    print(tagged)

[('Quiero', 'sps00'), ('una', 'di0fs0'), ('pizza', 'ncfs000'), ('con', 'sps00'), ('jamón', 'ncms000'), ('.', 'Fp')]
[('Quiero', 'sps00'), ('una', 'di0fs0'), ('pizza', 'ncfs000'), ('con', 'sps00'), ('queso', 'np0000l'), ('y', 'cc'), ('jamón', 'ncms000'), ('.', 'Fp')]
[('Quiero', 'da0mp0'), ('4', 'Z'), ('hamburguesas', 'ncmp000'), ('con', 'sps00'), ('tocino', 'np0000l'), ('.', 'Fp')]
[('Quiero', 'sps00'), ('una', 'di0fs0'), ('tortilla', 'ncfs000'), ('y', 'cc'), ('una', 'di0fs0'), ('cerveza', 'ncfs000'), ('.', 'Fp')]
[('Me', 'pp1cs000'), ('pones', 'vmip3s0'), ('un', 'di0ms0'), ('pollo', 'ncms000'), ('y', 'cc'), ('una', 'di0fs0'), ('ensalada', 'ncfs000'), ('.', 'Fp')]
[('Quiero', 'sps00'), ('una', 'di0fs0'), ('paella', 'ncfs000'), ('.', 'Fp')]
[('Quiero', 'sps00'), ('un', 'di0ms0'), ('bocadillo', 'ncms000'), ('.', 'Fp')]
[('Quiero', 'sps00'), ('una', 'di0fs0'), ('pizza', 'ncfs000'), ('.', 'Fp')]
[('Ponme', 'da0mp0'), ('3', 'Z'), ('sopas', 'ncfp000'), ('.', 'Fp')]
[('Quiero', 'sps00'), ('un

En la siguiente celda se define la gramática que permitirá extraer la comida y la cantidad de cada orden. Se ha podido identificar los siguientes patrones a partir de los tags de la celda anterior:
* Las cantidades cuando están escritas como palablas se identifican como determinante numérico (tag: dn...)
* Cuando las cantidades vienen dadas por números el tag es Z
* Por lo general los sustantivos de la comida vienen precedidos por un determinante indefinido (uno, un, una)
* Los ingredientes, que serán tomados en cuenta como parte de la comida, vienen precedidos de una preposición simple
* En caso de no cumplir ninguna de estas condiciones, los sustantivos que se encuentren solos serán clasificados como comida

In [20]:
# Gramática utilizada para extraer la comida y las cantidades
gramar = '''Q:           {<Z>|<dn.*>}                                                           # Q->cantidades 
            Food:        {(<di0..0> <n.*>|(<sps00><n.*>((<cc>|<Fc>)(<sn.*>|<n.*>))*)|<n.*>)}
         '''

Se realiza una prueba de la gramática con una de las ordenes de comida.

In [21]:
tokens = nltk.word_tokenize(food_orders[10])
tagged = hmm_tagger.tag(tokens)
print(tagged)
chunkParser = nltk.RegexpParser(gramar)
chunked = chunkParser.parse(tagged)
print(chunked)
for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Food'):
    print(subtree)

[('Por', 'sps00'), ('favor', 'ncms000'), (',', 'Fc'), ('me', 'pp1cs000'), ('das', 'vmip3s0'), ('tres', 'dn0cp0'), ('tacos', 'ncmp000'), ('.', 'Fp')]
(S
  (Food Por/sps00 favor/ncms000)
  ,/Fc
  me/pp1cs000
  das/vmip3s0
  (Q tres/dn0cp0)
  (Food tacos/ncmp000)
  ./Fp)
(Food Por/sps00 favor/ncms000)
(Food tacos/ncmp000)


En el resultado de la prueba se puede observar que se identifica de forma correcta a **tacos** como comida. Sin embargo, también se ha etiquetado incorrectamente a **por favor** como comida.
Para solventar este error se va a incluir un chinker, que retire la secuencia de preposición-sustantivo del chunker cuando venga seguida de un sigo de puntuación.

In [22]:
# Gramática utilizada para extraer la comida y las cantidades
gramar = '''
            Q:           {<Z>|<dn.*>|<di0.s0>}                                              # Q->cantidades 
            Food:        {(<sps00><n.*><Fc>*((<cc>|<Fc>)(<sn.*>|<n.*>))*)|<n.*>}            # Chunker de comidas
                         }<sps00><ncms000><Fc>{                                             # Chinker para retirar por favor
         '''

In [23]:
tokens = nltk.word_tokenize(food_orders[10])
tagged = hmm_tagger.tag(tokens)
print(tagged)
chunkParser = nltk.RegexpParser(gramar)
chunked = chunkParser.parse(tagged)
print(chunked)
for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Food'):
    print(subtree)

[('Por', 'sps00'), ('favor', 'ncms000'), (',', 'Fc'), ('me', 'pp1cs000'), ('das', 'vmip3s0'), ('tres', 'dn0cp0'), ('tacos', 'ncmp000'), ('.', 'Fp')]
(S
  Por/sps00
  favor/ncms000
  ,/Fc
  me/pp1cs000
  das/vmip3s0
  (Q tres/dn0cp0)
  (Food tacos/ncmp000)
  ./Fp)
(Food tacos/ncmp000)


A continuación se definen varias funciones encargadas de tareas específicas dentro del procesamiento de las ordenes de comida.

In [24]:
def get_chunks(sentence):
    '''Función que obtiene los chunks definidos por la gramática'''
    tokens = nltk.word_tokenize(sentence.lower())
    tagged = hmm_tagger.tag(tokens)
    chunked = chunkParser.parse(tagged)
    return chunked

In [25]:
def get_order_details(chunked, tag):
    '''Función que obtiene el valor según el tag de entrada'''
    chunk = [subtree.leaves() for subtree in chunked.subtrees(filter=lambda t: t.label() == tag)]
    #chunked.draw()
    return chunk

In [26]:
def map_num(q):
    '''Función que convierte las palabras que representan cantidades en números'''
    num = 1
    word_num = q[0][0][0] if q else 1
    if word_num == 'uno' or word_num == 'una': num = 1
    elif word_num == 'dos': num = 2
    elif word_num == 'tres': num = 3
    elif word_num == 'cuatro': num = 4
    elif word_num == 'cinco': num = 5
    elif word_num == 'seis': num = 6
    elif word_num == 'siete': num = 7
    elif word_num == 'ocho': num = 8
    elif word_num == 'nueve': num = 9
    elif word_num == 'diez': num = 10
    else:
        try:
            num = int(word_num)
        except:
            num = 1
    return num

In [27]:
def get_food(chunked):
    '''Función que extrae los sustantivos del chunk de comida'''
    food = []
    food_raw = get_order_details(chunked, 'Food')
    if food_raw:
        for item in food_raw:
            for word in item:
                if re.search('(^n|sn.e-SUJ)',word[1]):
                    food.append(word[0])
    return food

In [28]:
def build_order_dict(food, q):
    '''Función que construye un diccionario con la información de la orden'''
    order = {'comida' : food,
             'cantidad' : q
            }
    return order

In [29]:
def print_order_info(order, food, quantity):
    '''Función usada para imprimir de manera ordenada los detalles de la orden'''
    print('-------------------------------------')
    print(f'ORDER: {order}')
    print(f'FOOD:        {food}')
    print(f'QUANTITY:    {quantity}')
    print('-------------------------------------')

Ahora se va a utilizar las funciones ya definidas para ir porcesando una a una las ordenes de comida.
Además, se va a llenar dos listas. La primera va a almacenar los árboles con los IOB tags; y la segunda, los diccionarios con la información de comida y cantidad de cada orden

In [75]:
chunkParser = nltk.RegexpParser(gramar)
iob_tagged = []
orders_info = []
for order in food_orders:
    chunked = get_chunks(order)
    #print(chunked)
    iob_tagged.append(chunked)
    food = get_food(chunked)
    q = get_order_details(chunked, 'Q')
    n = map_num(q)
    orders_info.append(build_order_dict(food, n))
    print_order_info(order, food, n)

-------------------------------------
ORDER: Quiero una pizza con jamón.
FOOD:        ['pizza', 'jamón']
QUANTITY:    1
-------------------------------------
-------------------------------------
ORDER: Quiero una pizza con queso y jamón.
FOOD:        ['pizza', 'queso', 'jamón']
QUANTITY:    1
-------------------------------------
-------------------------------------
ORDER: Quiero 4 hamburguesas con tocino.
FOOD:        ['hamburguesas', 'tocino']
QUANTITY:    4
-------------------------------------
-------------------------------------
ORDER: Quiero una tortilla y una cerveza.
FOOD:        ['tortilla', 'cerveza']
QUANTITY:    1
-------------------------------------
-------------------------------------
ORDER: Me pones un pollo y una ensalada.
FOOD:        ['pollo', 'ensalada']
QUANTITY:    1
-------------------------------------
-------------------------------------
ORDER: Quiero una paella.
FOOD:        ['paella']
QUANTITY:    1
-------------------------------------
-----------------

In [76]:
orders_info

[{'comida': ['pizza', 'jamón'], 'cantidad': 1},
 {'comida': ['pizza', 'queso', 'jamón'], 'cantidad': 1},
 {'comida': ['hamburguesas', 'tocino'], 'cantidad': 4},
 {'comida': ['tortilla', 'cerveza'], 'cantidad': 1},
 {'comida': ['pollo', 'ensalada'], 'cantidad': 1},
 {'comida': ['paella'], 'cantidad': 1},
 {'comida': ['bocadillo'], 'cantidad': 1},
 {'comida': ['pizza'], 'cantidad': 1},
 {'comida': ['sopas'], 'cantidad': 3},
 {'comida': ['filete'], 'cantidad': 1},
 {'comida': ['tacos'], 'cantidad': 3},
 {'comida': ['orden', 'papitas', 'camotes'], 'cantidad': 1},
 {'comida': ['hamburguesas', 'cebollas', 'mayonessa', 'mostaza'],
  'cantidad': 4},
 {'comida': ['hotdogs', 'piña'], 'cantidad': 2},
 {'comida': ['alitas'], 'cantidad': 10},
 {'comida': ['ensalada', 'pollo'], 'cantidad': 1},
 {'comida': ['bolones', 'queso'], 'cantidad': 6},
 {'comida': ['tortillas'], 'cantidad': 12},
 {'comida': ['lasagna'], 'cantidad': 1},
 {'comida': ['bocadillos', 'anchoas'], 'cantidad': 3},
 {'comida': ['hambu

In [77]:
iob_tags = []
for order in iob_tagged:
    print(order)

(S
  quiero/sps00
  (Q una/di0fs0)
  (Food pizza/ncfs000)
  (Food con/sps00 jamón/ncms000)
  ./Fp)
(S
  quiero/sps00
  (Q una/di0fs0)
  (Food pizza/ncfs000)
  (Food con/sps00 queso/np0000l y/cc jamón/ncms000)
  ./Fp)
(S
  quiero/da0mp0
  (Q 4/Z)
  (Food hamburguesas/ncmp000)
  (Food con/sps00 tocino/np0000l)
  ./Fp)
(S
  quiero/sps00
  (Q una/di0fs0)
  (Food tortilla/ncfs000)
  y/cc
  (Q una/di0fs0)
  (Food cerveza/ncfs000)
  ./Fp)
(S
  me/pp1cs000
  pones/vmip3s0
  (Q un/di0ms0)
  (Food pollo/ncms000)
  y/cc
  (Q una/di0fs0)
  (Food ensalada/ncfs000)
  ./Fp)
(S quiero/sps00 (Q una/di0fs0) (Food paella/ncfs000) ./Fp)
(S quiero/sps00 (Q un/di0ms0) (Food bocadillo/ncms000) ./Fp)
(S quiero/sps00 (Q una/di0fs0) (Food pizza/ncfs000) ./Fp)
(S ponme/da0mp0 (Q 3/Z) (Food sopas/ncfp000) ./Fp)
(S quiero/sps00 (Q un/di0ms0) (Food filete/ncms000) ./Fp)
(S
  por/sps00
  favor/ncms000
  ,/Fc
  me/pp1cs000
  das/vmip3s0
  (Q tres/dn0cp0)
  (Food tacos/ncmp000)
  ./Fp)
(S
  me/pp1cs000
  puedes/vmip3s

## 4. Unigram Chunker

En la siguiente sección se va a entrenar un unigram chunker con los POS tags y IOB tags que se obtuvieron con el RegexpParser. El objetivo es utilizar los pos tags para poder asignar un IOB tag a las palabras.

In [70]:
from nltk.chunk.util import tree2conlltags,conlltags2tree

In [78]:
iob_tagged[:2]

[Tree('S', [('quiero', 'sps00'), Tree('Q', [('una', 'di0fs0')]), Tree('Food', [('pizza', 'ncfs000')]), Tree('Food', [('con', 'sps00'), ('jamón', 'ncms000')]), ('.', 'Fp')]),
 Tree('S', [('quiero', 'sps00'), Tree('Q', [('una', 'di0fs0')]), Tree('Food', [('pizza', 'ncfs000')]), Tree('Food', [('con', 'sps00'), ('queso', 'np0000l'), ('y', 'cc'), ('jamón', 'ncms000')]), ('.', 'Fp')])]

In [79]:
len(iob_tagged)

21

In [80]:
class UnigramChunker(nltk.ChunkParserI):
    
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

A continuación se va a entrenar el **unigram chunker**

In [81]:
train_sents = iob_tagged[:16]                     # Selecciono 17 de las 21 oraciones para el entrenamiento
test_sents = iob_tagged[16:]                      # Reservo las 4 restantes para prueba y evaluación
unigram_chunker = UnigramChunker(train_sents)     # Entreno el unigram chunker
print(unigram_chunker.evaluate(test_sents))       # Evaluo el chunker con las oraciones de prueba

ChunkParse score:
    IOB Accuracy:  84.6%%
    Precision:     84.6%%
    Recall:        84.6%%
    F-Measure:     84.6%%


El **unigram chunker** muestra un precission de 84.6%, lo que quiere decir que ha acertado el 84.6% de asignaciones de comida y cantidad.

Ahora se va a observar que aprendio el chunker mediante los tags que se asignaron a las oraciones de entrenamiento

In [82]:
postags = sorted(set(pos for sent in train_sents
                     for (word,pos) in sent.leaves()))
print(unigram_chunker.tagger.tag(postags))

[('Fc', 'O'), ('Fp', 'O'), ('Z', 'B-Q'), ('aq0mp0', 'O'), ('aq0mpp', 'O'), ('cc', 'O'), ('da0mp0', 'O'), ('di0fs0', 'B-Q'), ('di0ms0', 'B-Q'), ('dn0cp0', 'B-Q'), ('ncfp000', 'B-Food'), ('ncfs000', 'B-Food'), ('ncmp000', 'B-Food'), ('ncms000', 'I-Food'), ('np0000l', 'I-Food'), ('np0000p', 'B-Food'), ('pp1cs000', 'O'), ('rg', 'O'), ('sps00', 'O'), ('vmip3s0', 'O'), ('vmn0000', 'O')]


Y una nueva orden de comida, nunca antes vista por el chunker, para observar los resultados.

In [83]:
tokens = nltk.word_tokenize('Quiero tres donas con chocolate y nueces')
tagged = hmm_tagger.tag(tokens)
chunked = unigram_chunker.parse(tagged)
print(chunked)
chunked.draw()

(S
  Quiero/da0mp0
  (Q tres/dn0cp0)
  (Food donas/ncmp000)
  con/sps00
  (Food chocolate/ncms000)
  y/cc
  nueces/sn.e-SUJ)


Se puede ver que en esta oración se reconoce correctamente la comida: donas, chocolate. Sin embargo, no ha reconocido nueces como comida. Y al ser unigram, no se toma en cuenta las preposiciones o las comas; como se había definido en la gramática inicial.

Se va a construir el diccionario de resultados utilizando todo el corpus

In [84]:
chunkParser = unigram_chunker
iob_tagged = []
orders_info = []
for order in food_orders:
    chunked = get_chunks(order)
    #print(chunked)
    iob_tagged.append(chunked)
    food = get_food(chunked)
    q = get_order_details(chunked, 'Q')
    n = map_num(q)
    orders_info.append(build_order_dict(food, n))
    #print_order_info(order, food, n)
for i in range(len(orders_info)):
    print(i, orders_info[i])

0 {'comida': ['pizza', 'jamón'], 'cantidad': 1}
1 {'comida': ['pizza', 'queso', 'jamón'], 'cantidad': 1}
2 {'comida': ['hamburguesas', 'tocino'], 'cantidad': 4}
3 {'comida': ['tortilla', 'cerveza'], 'cantidad': 1}
4 {'comida': ['pollo', 'ensalada'], 'cantidad': 1}
5 {'comida': ['paella'], 'cantidad': 1}
6 {'comida': ['bocadillo'], 'cantidad': 1}
7 {'comida': ['pizza'], 'cantidad': 1}
8 {'comida': ['sopas'], 'cantidad': 3}
9 {'comida': ['filete'], 'cantidad': 1}
10 {'comida': ['favor', 'tacos'], 'cantidad': 3}
11 {'comida': ['orden', 'papitas', 'camotes'], 'cantidad': 1}
12 {'comida': ['hamburguesas', 'cebollas', 'mayonessa', 'mostaza'], 'cantidad': 4}
13 {'comida': ['hotdogs', 'piña'], 'cantidad': 2}
14 {'comida': ['alitas'], 'cantidad': 10}
15 {'comida': ['favor', 'ensalada', 'pollo'], 'cantidad': 1}
16 {'comida': ['bolones', 'queso'], 'cantidad': 6}
17 {'comida': ['tortillas'], 'cantidad': 12}
18 {'comida': ['lasagna'], 'cantidad': 1}
19 {'comida': ['bocadillos', 'anchoas'], 'cantida

Cuando se utiliza el **unigram chunker** sobre el corpus de entrenamiento, surgen las siuientes observaciones:
* En la orden 10 y 15, dada la natraleza del unigram tagger, se ha perdido el contexto y se reconoce a la palabra *favor* como comida.

In [85]:
iob_tagged[10].__repr__()

"Tree('S', [('por', 'sps00'), Tree('Food', [('favor', 'ncms000')]), (',', 'Fc'), ('me', 'pp1cs000'), ('das', 'vmip3s0'), Tree('Q', [('tres', 'dn0cp0')]), Tree('Food', [('tacos', 'ncmp000')]), ('.', 'Fp')])"

Se puede verificar que *favor* ha sido etiquetado como 'ncms00' y en los tags que aprendió el **unigram chunker** el IOB tag correspondiente es 'I-Food'.

## 5. Bigram Chunker

Ahora se va a definir y evaluar un **bigram chunker**

In [86]:
class BigramChunker(nltk.ChunkParserI):
    
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [87]:
train_sents = iob_tagged[:16]
test_sents = iob_tagged[16:]
bigram_chunker = BigramChunker(train_sents)
print(bigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  88.5%%
    Precision:    100.0%%
    Recall:        76.9%%
    F-Measure:     87.0%%


Se puede ver que el porcentaje de acierto en las asignaciones de comida y cantidad ha subido al 100%. Y en general el F-meassure también ha subido al 87%. Los aciertos totales representados por el accuracy también han subido al 88.5%.

In [89]:
tokens = nltk.word_tokenize('Quiero tres donas con chocolate y nueces')
tagged = hmm_tagger.tag(tokens)
chunked = bigram_chunker.parse(tagged)
print(chunked)
chunked.draw()

(S
  Quiero/da0mp0
  (Q tres/dn0cp0)
  (Food donas/ncmp000)
  con/sps00
  (Food chocolate/ncms000)
  y/cc
  nueces/sn.e-SUJ)


In [90]:
chunkParser = unigram_chunker
iob_tagged = []
orders_info = []
for order in food_orders:
    chunked = get_chunks(order)
    #print(chunked)
    iob_tagged.append(chunked)
    food = get_food(chunked)
    q = get_order_details(chunked, 'Q')
    n = map_num(q)
    orders_info.append(build_order_dict(food, n))
    #print_order_info(order, food, n)
for i in range(len(orders_info)):
    print(i, orders_info[i])

0 {'comida': ['pizza', 'jamón'], 'cantidad': 1}
1 {'comida': ['pizza', 'queso', 'jamón'], 'cantidad': 1}
2 {'comida': ['hamburguesas', 'tocino'], 'cantidad': 4}
3 {'comida': ['tortilla', 'cerveza'], 'cantidad': 1}
4 {'comida': ['pollo', 'ensalada'], 'cantidad': 1}
5 {'comida': ['paella'], 'cantidad': 1}
6 {'comida': ['bocadillo'], 'cantidad': 1}
7 {'comida': ['pizza'], 'cantidad': 1}
8 {'comida': ['sopas'], 'cantidad': 3}
9 {'comida': ['filete'], 'cantidad': 1}
10 {'comida': ['favor', 'tacos'], 'cantidad': 3}
11 {'comida': ['orden', 'papitas', 'camotes'], 'cantidad': 1}
12 {'comida': ['hamburguesas', 'cebollas', 'mayonessa', 'mostaza'], 'cantidad': 4}
13 {'comida': ['hotdogs', 'piña'], 'cantidad': 2}
14 {'comida': ['alitas'], 'cantidad': 10}
15 {'comida': ['favor', 'ensalada', 'pollo'], 'cantidad': 1}
16 {'comida': ['bolones', 'queso'], 'cantidad': 6}
17 {'comida': ['tortillas'], 'cantidad': 12}
18 {'comida': ['lasagna'], 'cantidad': 1}
19 {'comida': ['bocadillos', 'anchoas'], 'cantida

In [96]:
from tag_util import backoff_tagger

ModuleNotFoundError: No module named 'tag_util'

In [97]:
def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]

class TagChunker(ChunkParserI):
    
    def __init__(self, train_chunks, tagger_classes=[UnigramTagger, BigramTagger]):
        train_sents = conll_tag_chunks(train_chunks)
        self.tagger = backoff_tagger(train_sents, tagger_classes)
    
    def parse(self, tagged_sent):
        if not tagged_sent: return None
        (words, tags) = zip(*tagged_sent)
        chunks = self.tagger.tag(tags)
        wtc = zip(words, chunks)
        return conlltags2tree([(w,t,c) for (w,(t,c)) in wtc])

In [95]:
train_sents = iob_tagged[:16]
test_sents = iob_tagged[16:]
chunker = TagChunker(train_sents)

NameError: name 'backoff_tagger' is not defined

## Naive Bayes Chunker

In [92]:
from nltk.classify import megam
megam.config_megam()

LookupError: 

===========================================================================
NLTK was unable to find the megam file!
Use software specific configuration paramaters or set the MEGAM environment variable.

  For more information on megam, see:
    <http://www.umiacs.umd.edu/~hal/megam/index.html>
===========================================================================

In [93]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train(
            train_set, algorithm='megam', trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [91]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

LookupError: 

===========================================================================
NLTK was unable to find the megam file!
Use software specific configuration paramaters or set the MEGAM environment variable.

  For more information on megam, see:
    <http://www.umiacs.umd.edu/~hal/megam/index.html>
===========================================================================